In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy
from itertools import product

**Day 1**

In [13]:
df = pd.read_csv('input1.txt', header=None)
df.head()

,0
0,141
1,140
2,160
3,161
4,162


Part 1

In [4]:
sum((df.diff() > 0).values)

array([1692])

Part 2

In [7]:
sum((df.rolling(3).sum().diff() > 0).values)

array([1724])

**Day 2**

In [43]:
df = pd.read_csv('input2.txt', header=None).loc[:, 0].str.split(' ', expand=True).set_axis(['direction', 'steps'], axis=1).astype({'steps': int})
df.head()

,direction,steps
0,forward,3
1,down,9
2,forward,6
3,down,3
4,forward,8


Part 1

In [44]:
df[df['direction']=='forward'].steps.sum() * (df[df['direction']=='down'].steps.sum() - df[df['direction']=='up'].steps.sum())

2073315

Part 2

In [58]:
df[df['direction']=='forward'].steps.sum() * (df['direction'].map({'forward': 1, 'up': 0, 'down': 0}) * (df['direction'].map({'up': -1, 'down': 1, 'forward': 0}) * df['steps']).cumsum() * df['steps']).sum()

1840311528

**Day 3**

In [3]:
df = pd.read_csv('input3.txt', header=None, dtype=str)[0].str.split('', expand=True).loc[:, 1:12]
df.head()

,1,2,3,4,5,6,7,8,9,10,11,12
0,0,1,1,1,1,1,1,1,1,1,0,1
1,0,1,1,0,0,0,0,1,0,1,1,1
2,0,0,0,0,0,0,0,0,0,1,0,1
3,0,0,0,1,1,1,1,0,0,0,0,1
4,0,1,1,0,1,1,0,1,0,0,0,0


Part 1

In [4]:
(df.mode().squeeze().astype(int) * (2 ** np.arange(12)[::-1])).sum() * \
(df.mode().squeeze().map({'0': '1', '1': '0'}).astype(int) * (2 ** np.arange(12)[::-1])).sum()

4174964

Part 2

In [23]:
o2_gen_rating, co2_gen_rating = df.copy(deep=True), df.copy(deep=True)
for col in range(1, 13):
    if len(o2_gen_rating) > 1:
        temp = o2_gen_rating[col].value_counts()
        if len(np.unique(temp.values)) == 1:
            temp = '1'
        else:
            temp = temp.keys()[0]
        o2_gen_rating = o2_gen_rating[o2_gen_rating[col]==temp]
    if len(co2_gen_rating) > 1:
        temp = co2_gen_rating[col].value_counts()
        if len(np.unique(temp.values)) == 1:
            temp = '0'
        else:
            temp = temp.keys()[1]
        co2_gen_rating = co2_gen_rating[co2_gen_rating[col]==temp]

(o2_gen_rating.squeeze().astype(int) * (2 ** np.arange(12)[::-1])).sum() * \
(co2_gen_rating.squeeze().astype(int) * (2 ** np.arange(12)[::-1])).sum()

4474944

**Day 4**

In [170]:
callout_numbers = np.genfromtxt('input4.txt', delimiter=',', dtype=int, max_rows=1)
callout_numbers

array([25,  8, 32, 53, 22, 94, 55, 80, 33,  4, 63, 14, 60, 95, 31, 89, 30,
        5, 47, 66, 84, 70, 17, 74, 99, 82, 21, 35, 64,  2, 76,  9, 90, 56,
       78, 28, 51, 86, 49, 98, 29, 96, 23, 58, 52, 75, 41, 50, 13, 72, 92,
       83, 62, 37, 18, 11, 34, 71, 91, 85, 27, 12, 24, 73,  7, 77, 10, 93,
       15, 61,  3, 46, 16, 97,  1, 57, 65, 40,  0, 48, 69,  6, 20, 68, 19,
       45, 42, 79, 88, 44, 26, 38, 36, 54, 81, 59, 43, 87, 39, 67])

In [180]:
bingo_boards = []
for line_number in range(3, len(open('input4.txt', 'r').readlines()), 6):
    bingo_boards.append(np.genfromtxt('input4.txt', dtype=int, skip_header=line_number-1, max_rows=5))
len(bingo_boards)

100

Part 1

In [194]:
bingo_boards_copy = deepcopy(bingo_boards)

winning_score = 0
for call in callout_numbers:
    for board in bingo_boards_copy:
        board[board==call] = -1
        if np.all(np.diff(board, axis=0)==0, axis=0).any() or np.all(np.diff(board, axis=1)==0, axis=1).any():
            if (board.sum() + (board == -1).sum()) * call > winning_score :
                winning_score = (board.sum() + (board == -1).sum()) * call
            break
    if winning_score != 0:
        break

winning_score

50008

Part 2

In [193]:
bingo_boards_copy = deepcopy(bingo_boards)

winning_board_positions, last_winning_call = [], 0
for call in callout_numbers:
    for board_num in range(len(bingo_boards_copy)):
        if board_num in winning_board_positions:
            continue
        board = bingo_boards_copy[board_num]
        board[board==call] = -1
        if np.all(np.diff(board, axis=0)==0, axis=0).any() or np.all(np.diff(board, axis=1)==0, axis=1).any():
            last_winning_call = call
            winning_board_positions.append(board_num)

last_winning_board = bingo_boards_copy[winning_board_positions[-1]]
winning_score = (last_winning_board.sum() + (last_winning_board == -1).sum()) * last_winning_call
winning_score

17408

**Day 5**

In [107]:
df = pd.read_csv('input5.txt', header=None, sep=' -> ').set_axis([1, 2], axis=1)
df[['x1', 'y1']] = df[1].str.split(',', expand=True).astype(int)
df[['x2', 'y2']] = df[2].str.split(',', expand=True).astype(int)

df.head()

<ipython-input-107-4b4a66af2008>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('input5.txt', header=None, sep=' -> ').set_axis([1, 2], axis=1)


,1,2,x1,y1,x2,y2
0,"822,976","822,117",822,976,822,117
1,"387,419","387,468",387,419,387,468
2,"659,275","318,616",659,275,318,616
3,"535,940","684,940",535,940,684,940
4,"294,427","294,88",294,427,294,88


Part 1

In [138]:
vertical_horizontal_lines = df.loc[(df.x1==df.x2) | (df.y1==df.y2)]
min_x, max_x = vertical_horizontal_lines[['x1', 'x2']].min().min(), vertical_horizontal_lines[['x1', 'x2']].max().max()
min_y, max_y = vertical_horizontal_lines[['y1', 'y2']].min().min(), vertical_horizontal_lines[['y1', 'y2']].max().max()

field = np.zeros((max_x-min_x, max_y-min_y))
for _, line in vertical_horizontal_lines.iterrows():
    field[min(line.x1, line.x2)-min_x:max(line.x1, line.x2)-min_x+1,
          min(line.y1, line.y2)-min_y:max(line.y1, line.y2)-min_y+1] += 1
            
(field>1).sum()

5294

Part 2

In [183]:
min_x, max_x = df[['x1', 'x2']].min().min(), df[['x1', 'x2']].max().max()
min_y, max_y = df[['y1', 'y2']].min().min(), df[['y1', 'y2']].max().max()

field = np.zeros((max_x-min_x, max_y-min_y))
for _, line in df.iterrows():
    if line.x1 == line.x2 or line.y1 == line.y2:
        field[min(line.x1, line.x2)-min_x:max(line.x1, line.x2)-min_x+1,
              min(line.y1, line.y2)-min_y:max(line.y1, line.y2)-min_y+1] += 1
    else:
        if line.x1 < line.x2:
            if line.y1 < line.y2:
                np.fill_diagonal(field[line.x1-min_x:line.x2-min_x+1, line.y1-min_y:line.y2-min_y+1],
                                field[line.x1-min_x:line.x2-min_x+1, line.y1-min_y:line.y2-min_y+1].diagonal()+1)
            else:
                np.fill_diagonal(np.flipud(field[line.x1-min_x:line.x2-min_x+1, line.y2-min_y:line.y1-min_y+1]),
                                np.flipud(field[line.x1-min_x:line.x2-min_x+1, line.y2-min_y:line.y1-min_y+1]).diagonal()+1)
        else:
            if line.y1 < line.y2:
                np.fill_diagonal(np.flipud(field[line.x2-min_x:line.x1-min_x+1, line.y1-min_y:line.y2-min_y+1]),
                                np.flipud(field[line.x2-min_x:line.x1-min_x+1, line.y1-min_y:line.y2-min_y+1]).diagonal()+1)
            else:
                np.fill_diagonal(field[line.x2-min_x:line.x1-min_x+1, line.y2-min_y:line.y1-min_y+1],
                        field[line.x2-min_x:line.x1-min_x+1, line.y2-min_y:line.y1-min_y+1].diagonal()+1)

(field>1).sum()

21698

**Day 6**

In [186]:
lantern_fish = np.genfromtxt('input6.txt', delimiter=',', dtype=int)
lantern_fish[:5]

array([3, 5, 1, 5, 3])

Part 1

In [234]:
def offsprings(initial_days, total_days, known_results):
    if (initial_days, total_days) in known_results:
        return known_results[(initial_days, total_days)]
    count = 0
    for child_day in range(initial_days, total_days, 7):
        count += (1 + offsprings(child_day+9, total_days, known_results))
    known_results[(initial_days, total_days)] = count
    return count

total_fish = len(lantern_fish)
known_results = {}
for inits in lantern_fish:
    total_fish += offsprings(inits, 80, known_results)
total_fish

351188

Part 2

In [239]:
total_fish = len(lantern_fish)
known_results = {}
for inits in lantern_fish:
    total_fish += offsprings(inits, 256, known_results)
total_fish

1595779846729

**Day 7**

In [242]:
crab_positions = np.genfromtxt('input7.txt', delimiter=',', dtype=int)
crab_positions[:5]

array([1101,    1,   29,   67, 1102])

Part 1

In [265]:
def fuel(x):
    return abs(crab_positions - x).sum()

min_fuel = 0
for dist in range(min(crab_positions), max(crab_positions)+1):
    new_fuel = fuel(dist)
    if min_fuel == 0 or new_fuel < min_fuel:
        min_fuel = new_fuel
        
min_fuel

348664

Part 2

In [267]:
def new_fuel_calc(x):
    old_calc = abs(crab_positions - x)
    return ((old_calc * (old_calc + 1)) // 2).sum()

min_fuel = 0
for dist in range(min(crab_positions), max(crab_positions)+1):
    new_fuel = new_fuel_calc(dist)
    if min_fuel == 0 or new_fuel < min_fuel:
        min_fuel = new_fuel
        
min_fuel

100220525

**Day 8**

In [273]:
df = pd.read_csv('input8.txt', header=None, sep=' ').drop(10, axis=1)
df.head()

,0,1,2,3,4,5,6,7,8,9,11,12,13,14
0,fgcae,ebafc,cabdef,eg,abecfg,abgfed,feg,gafdc,bceg,ebgcadf,defagbc,faecg,cfdag,gecb
1,eagd,cad,fgadbc,aefdcg,dcebfg,fcegd,cbeaf,ad,dbgfeca,defca,cfdeg,gdcabf,fcgde,afgced
2,gfdeca,aeb,eb,fbdag,eafdc,adfbe,cefdab,bdaegcf,efbc,ecadgb,eb,dbafe,eab,faecd
3,facbdge,efdg,gcafd,daegc,caegb,aecfdb,ade,ed,gafbdc,dgfeac,de,abcdef,faedcg,dfgca
4,dcaegbf,bc,dcgeab,cbgad,cebg,cdfaeb,fbadg,acdge,cefdga,cdb,gdcfae,gbec,gdace,cgadfe


Part 1

In [279]:
output_cardinality = df.iloc[:, -4:].apply(lambda col: col.str.len())

total = 0
unique_digit_sequence = [2, 3, 4, 7] # corresponding to the digits 1, 4, 7 and 8
for col in output_cardinality:
    total += len(output_cardinality[output_cardinality[col].isin(unique_digit_sequence)])

total

278

Part 2

In [319]:
def find_digits(input_row):
    wire_sequence = input_row.iloc[:-4].apply(lambda one_seq: frozenset(one_seq))
    _4digit_number = input_row.iloc[-4:].apply(lambda one_digit: frozenset(one_digit))
    
    wire_seq_length = wire_sequence.apply(len)
    
    wire_digit_mapping = {1: wire_sequence.loc[np.where(wire_seq_length == 2)].values[0], 
                          4: wire_sequence.loc[np.where(wire_seq_length == 4)].values[0],
                          7: wire_sequence.loc[np.where(wire_seq_length == 3)].values[0],
                          8: wire_sequence.loc[np.where(wire_seq_length == 7)].values[0]}
    
    for seq in wire_sequence.loc[np.where(wire_seq_length == 5)].values:
        if seq | wire_digit_mapping[4] == wire_digit_mapping[8]:
            wire_digit_mapping[2] = seq
        elif len(seq ^ wire_digit_mapping[7]) == 2:
            wire_digit_mapping[3] = seq
        else:
            wire_digit_mapping[5] = seq
            
    wire_digit_mapping[9] = wire_digit_mapping[3] | wire_digit_mapping[5]

    for seq in wire_sequence.loc[np.where(wire_seq_length == 6)].values:
        if seq == wire_digit_mapping[9]:
            continue
        elif seq | wire_digit_mapping[1] == seq:
            wire_digit_mapping[0] = seq
        else:
            wire_digit_mapping[6] = seq
    
    digit_wire_mapping = dict(zip(wire_digit_mapping.values(), wire_digit_mapping.keys()))
    
    return (_4digit_number.map(digit_wire_mapping) * (10**np.arange(3, -1, -1))).sum()

df.apply(find_digits, axis=1).sum()

986179

**Day 9**

In [362]:
heightmap = pd.read_csv('input9_test.txt', header=None, dtype=str)[0].str.split('', expand=True).iloc[:, 1:-1].astype(int).values
heightmap

array([[2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
       [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
       [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
       [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
       [9, 8, 9, 9, 9, 6, 5, 6, 7, 8]])

Part 1

In [367]:
expanded_heightmap = np.pad(heightmap, ((1, 1), (1, 1)), 'maximum')

results = np.random.choice(a=[True], size=(heightmap.shape[0], heightmap.shape[1]))
for shift in [1, -1]:
    results &= (heightmap < np.roll(expanded_heightmap, (shift, 0), axis=(0, 1))[1:-1, 1:-1])
    results &= (heightmap < np.roll(expanded_heightmap, (0, shift), axis=(0, 1))[1:-1, 1:-1])
(results * heightmap).sum() + results.sum()

15

Part 2

In [368]:
expanded_heightmap = np.pad(heightmap, ((1, 1), (1, 1)), 'maximum')

results_1 = np.random.choice(a=[False], size=(heightmap.shape[0], heightmap.shape[1]))
for shift in [1, -1]:
    results_1 |= (heightmap < np.roll(expanded_heightmap, (shift, 0), axis=(0, 1))[1:-1, 1:-1])
    results_1 |= (heightmap < np.roll(expanded_heightmap, (0, shift), axis=(0, 1))[1:-1, 1:-1])
results_1

array([[ True,  True, False, False, False,  True,  True,  True,  True,
         True],
       [ True, False,  True,  True,  True, False,  True, False,  True,
         True],
       [False,  True,  True,  True,  True,  True, False,  True, False,
         True],
       [ True,  True,  True,  True,  True, False,  True,  True,  True,
        False],
       [False,  True, False, False, False,  True,  True,  True,  True,
         True]])

In [387]:
def coords_boundary(left, top, bottom, right, highest_row, highest_col):
    left, top = 0 if left==0 else left-1, 0 if top==0 else top-1
    bottom, right = bottom if bottom==highest_row else bottom+1, right if right==highest_col else right+1
    return [left, top, bottom, right]

three_largest_basins = []

highest_row, highest_col = results.shape[0], results.shape[1]
for coords in np.array(np.where(results == True)).T[:1, :]:    
    top, left = 0 if coords[0]==0 else coords[0]-1, 0 if coords[1]==0 else coords[1]-1
    bottom, right = coords[0] if coords[0]==results.shape[0] else coords[0]+1, coords[1] if coords[1]==results.shape[1] else coords[1]+1
    print(left, top, bottom, right)
    
    old_basin_size = results_1[top:bottom+1, left:right+1].sum()
    print(left, top, bottom, right, old_basin_size, results_1[top:bottom+1, left:right+1])
    left, top, bottom, right = coords_boundary(left, top, bottom, right, highest_row, highest_col)
    new_basin_size = results_1[top:bottom+1, left:right+1].sum()
#     print(left, top, bottom, right, old_basin_size, new_basin_size)
    while new_basin_size > old_basin_size:
        left, top, bottom, right = coords_boundary(left, top, bottom, right, highest_row, highest_col)
#         print(left, top, bottom, right, old_basin_size, new_basin_size)
        old_basin_size = new_basin_size
        new_basin_size = results_1[top:bottom+1, left:right+1].sum()
        
    if len(three_largest_basins) < 3:
        three_largest_basins.append(new_basin_size)
    elif new_basin_size > min(three_largest_basins):
        three_largest_basins.pop(three_largest_basins.index(min(three_largest_basins)))
        three_largest_basins.append(new_basin_size)
        
# np.array(three_largest_basins).prod()
three_largest_basins

0 0 1 2
0 0 1 2 4 [[ True  True False]
 [ True False  True]]


[35]